In [1]:
import pandas as pd

/Users/Vivek/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/Vivek/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import wrds
from datetime import date, timedelta
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import pandas as pd
import os
from tqdm import tqdm

In [3]:
conn = wrds.Connection(wrds_username='vivek12', wrds_password='Powerplay@ec3')

Loading library list...
Done


In [4]:
folders = os.listdir("MAEC_Dataset")

In [5]:
change = []
def get_change(parsed, ticker):
    start_date = parsed - relativedelta(days=1)
    end_date = parsed + relativedelta(days=1)

    # Query CRSP database for stock prices
    crsp_query = f"""
        SELECT date, prc
        FROM crsp.dsf
        WHERE permco IN (
            SELECT permco
            FROM crsp.dse
            WHERE ticker = '{ticker}'
            )
        AND date BETWEEN '{start_date}' AND '{end_date}'
    """

    # Execute the query and retrieve data
    data = conn.raw_sql(crsp_query)


    # Convert the retrieved data to a pandas DataFrame
    df = pd.DataFrame(data)

    ind = abs(df['date'] - parsed.date()).idxmin()
    old = df.loc[ind-1]['prc']
    new = df.loc[ind]['prc']
    chg = (new-old)/old
    return (chg*100)

In [13]:
initial = {'Date': [], 'Ticker':[]}
df = pd.DataFrame(initial)
df.to_csv("change.csv", index=False)

In [17]:
years = []
tickers = []
changes = []
initial = {'Date': [], 'Ticker': [], 'Change':[]}
df = pd.read_csv("change.csv")
for i in tqdm(range(len(folders))):
    folder = folders[i]
    year, ticker = folder.split('_')
#    if ticker in list(df['Ticker']):
#        continue
    try:
        parsed = parse(year)
    except:
        print("No")
        continue
    true_date = parsed.strftime('%Y-%m-%d')
#    try:
#        changes.append(get_change(true_date, ticker))
#    except:
#        print(f"Didn't work {folder}")
#        continue
    years.append(true_date)
    tickers.append(ticker)
    if i % 100 == 0 and i!= 0:
        df = pd.read_csv("change.csv")
        data = {'Date': years, 'Ticker': tickers}
        df2 = pd.DataFrame(data)
        years = []
        tickers = []
        changes = []
        df3 = pd.concat([df, df2])
        df3.to_csv("change.csv", index = False) 

 73%|██████████████████████████▏         | 2501/3444 [00:00<00:00, 12498.08it/s]

No


100%|████████████████████████████████████| 3444/3444 [00:00<00:00, 12184.12it/s]
